In [ ]:
# @title 1. Setup Lingkungan, Instalasi Dependensi, Download model
import os
import subprocess

print("Memulai proses setup...")

# Pindah ke direktori utama
%cd /content

# Hapus direktori ComfyUI jika sudah ada untuk memastikan instalasi bersih
if os.path.exists('/content/ComfyUI'):
    print("Menghapus instalasi ComfyUI sebelumnya...")
    subprocess.run('rm -rf /content/ComfyUI', shell=True, check=True)

# Clone repository ComfyUI dan custom node untuk GGUF
print("Cloning repository ComfyUI dan custom node ComfyUI-GGUF...")
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI --quiet
!git clone https://github.com/city96/ComfyUI-GGUF.git /content/ComfyUI/custom_nodes/ComfyUI-GGUF --quiet

# Pindah ke direktori ComfyUI
%cd /content/ComfyUI

print("Menginstal dependensi yang diperlukan...")
# Menggunakan versi torch yang kompatibel dengan Colab T4 (CUDA 12.1)
#!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121 -q
!pip install -r requirements.txt accelerate gradio gguf -q
print("Instalasi dependensi selesai.")

# Instal aria2 untuk mempercepat unduhan
!apt update -qqy
!apt upgrade -qqy
!apt install -qqy aria2

print("Setup dependensi selesai!")

# Direktori untuk menyimpan model
UNET_DIR = "/content/ComfyUI/models/unet"
CLIP_DIR = "/content/ComfyUI/models/clip"
VAE_DIR = "/content/ComfyUI/models/vae"

# Membuat direktori jika belum ada
os.makedirs(UNET_DIR, exist_ok=True)
os.makedirs(CLIP_DIR, exist_ok=True)
os.makedirs(VAE_DIR, exist_ok=True)

# Daftar model yang akan diunduh
models_to_download = {
    "unet": {
        "url": "https://huggingface.co/calcuis/cosmos-predict2-gguf/resolve/main/cosmos-predict2-2b-t2i-q8_0.gguf",
        "dir": UNET_DIR,
        "name": "cosmos-predict2-2b-t2i-q8_0.gguf"
    },
    "clip": {
        "url": "https://huggingface.co/chatpig/t5-v1_1-xxl-old-encoder-gguf/resolve/main/t5-v1_1-xxl-old-encoder-q4_0.gguf",
        "dir": CLIP_DIR,
        "name": "t5-v1_1-xxl-old-encoder-q4_0.gguf"
    },
    "vae": {
        "url": "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors",
        "dir": VAE_DIR,
        "name": "wan_2.1_vae.safetensors"
    }
}

# Fungsi untuk mengunduh model menggunakan aria2
def download_model(model_info):
    print(f"Mengunduh {model_info['name']}...")
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d \"{model_info['dir']}\" -o \"{model_info['name']}\" \"{model_info['url']}\""
    subprocess.run(command, shell=True, check=True)
    print(f"Selesai mengunduh {model_info['name']}.")

# Lakukan pengunduhan
for model_type, model_data in models_to_download.items():
    download_model(model_data)

print("\nSemua model berhasil diunduh!")

In [ ]:
# @title 2. Inisialisasi dan Jalankan Antarmuka Gradio (Diperbaiki v2)

import sys
import torch
import random
import time
from PIL import Image
import numpy as np
import gradio as gr
import os
import logging
import asyncio

# Pindah ke direktori kerja yang benar, ini sangat penting
%cd /content/ComfyUI

# Tambahkan direktori ComfyUI ke path Python agar impor berhasil
if '/content/ComfyUI' not in sys.path:
    sys.path.append('/content/ComfyUI')

# --- LANGKAH 1: Definisikan Fungsi Setup Asinkron ---
# Kita bungkus semua proses loading dalam satu fungsi async
async def setup_and_load_models():
    # <<< PERBAIKAN UTAMA: Pindahkan SEMUA impor ComfyUI ke dalam fungsi async
    # Ini memastikan direktori kerja sudah benar sebelum impor dijalankan.
    print("Mengimpor fungsi dan class dari ComfyUI...")
    from nodes import NODE_CLASS_MAPPINGS, KSamplerAdvanced, VAEDecode, CLIPTextEncode, EmptyLatentImage, VAELoader, load_custom_node
    import comfy.samplers

    print("Memuat custom nodes dari ComfyUI-GGUF...")
    # Gunakan 'await' untuk menjalankan fungsi asinkron
    await load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI-GGUF")
    print("-> Node kustom berhasil dimuat dan didaftarkan.")

    # --- LANGKAH 2: Inisialisasi Class Node ---
    print("\nMenginisialisasi class-class node...")
    try:
        # Kita ambil class dari mapping setelah custom node berhasil dimuat
        UnetLoaderGGUF_class = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]
        CLIPLoaderGGUF_class = NODE_CLASS_MAPPINGS["CLIPLoaderGGUF"]

        # Buat instance dari class tersebut
        unet_loader_gguf = UnetLoaderGGUF_class()
        clip_loader_gguf = CLIPLoaderGGUF_class()
        vae_loader = VAELoader()
        print("-> Semua class loader berhasil diinisialisasi.")
    except KeyError as e:
        print(f"!!! ERROR: Gagal menemukan class node yang diperlukan: '{e}'. Pastikan ComfyUI-GGUF terinstal dengan benar.")
        raise e

    # --- LANGKAH 3: Muat Model ke Memori ---
    print("\nMemuat model Unet, CLIP, dan VAE ke memori...")
    # Pastikan file model ada di direktori yang benar (misal: /content/ComfyUI/models/checkpoints/)
    # ComfyUI-GGUF biasanya mencari model di /content/ComfyUI/models/gguf/
    unet_model = unet_loader_gguf.load_unet("cosmos-predict2-2b-t2i-q8_0.gguf")[0]
    clip_model = clip_loader_gguf.load_clip("t5-v1_1-xxl-old-encoder-q4_0.gguf", "cosmos")[0]
    vae_model = vae_loader.load_vae("wan_2.1_vae.safetensors")[0]
    print("-> Semua model utama berhasil dimuat!")

    return unet_model, clip_model, vae_model, NODE_CLASS_MAPPINGS, comfy.samplers

# Definisikan variabel global untuk menampung model yang sudah dimuat
unet_model, clip_model, vae_model, NODE_CLASS_MAPPINGS, samplers = [None] * 5

# --- Jalankan proses setup asinkron ---
async def main():
    global unet_model, clip_model, vae_model, NODE_CLASS_MAPPINGS, samplers
    try:
        unet_model, clip_model, vae_model, NODE_CLASS_MAPPINGS, samplers = await setup_and_load_models()
        logging.getLogger().setLevel(logging.WARNING)
    except Exception as e:
        print(f"\n!!! GAGAL PADA TAHAP SETUP: {e} !!!")
        import traceback
        traceback.print_exc()

# Jalankan fungsi main di event loop
# Ini cara yang lebih robust untuk menjalankan async code di script
await main()


# --- LANGKAH 4: Definisikan Fungsi Generate dan UI Gradio ---
def generate_image(positive_prompt, negative_prompt, width, height, steps, cfg, seed, sampler_name, scheduler):
    # Pastikan model sudah dimuat sebelum generate
    if not unet_model:
        print("!!! ERROR: Model belum dimuat. Silakan restart runtime dan jalankan lagi. !!!")
        return None

    print("\n--- Memulai Proses Generate Gambar ---")
    try:
        with torch.inference_mode():
            # Ambil class dari mapping yang sudah diperbarui
            CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]
            EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]
            KSamplerAdvanced = NODE_CLASS_MAPPINGS["KSamplerAdvanced"]
            VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]

            clip_text_encode = CLIPTextEncode()
            empty_latent_image = EmptyLatentImage()
            k_sampler_advanced = KSamplerAdvanced()
            vae_decode = VAEDecode()

            print("-> Encoding prompt...")
            positive_conditioning = clip_text_encode.encode(clip_model, positive_prompt)[0]
            negative_conditioning = clip_text_encode.encode(clip_model, negative_prompt)[0]

            print(f"-> Membuat latent kosong {width}x{height}...")
            latent_image = empty_latent_image.generate(width, height, 1)[0]

            if seed == -1:
                seed = random.randint(0, 2**64 - 1)
            print(f"-> Menggunakan Seed: {seed}")

            print(f"-> Menjalankan KSampler...")
            samples = k_sampler_advanced.sample(
                model=unet_model,
                add_noise="enable",
                noise_seed=seed,
                steps=steps,
                cfg=cfg,
                sampler_name=sampler_name,
                scheduler=scheduler,
                positive=positive_conditioning,
                negative=negative_conditioning,
                latent_image=latent_image,
                start_at_step=0,
                end_at_step=9999,
                return_with_leftover_noise="disable"
            )[0]

            print("-> Decoding menjadi gambar...")
            decoded_image_tensor = vae_decode.decode(vae_model, samples)[0]

            image_np = decoded_image_tensor.cpu().numpy()
            image_np_uint8 = (image_np * 255).astype(np.uint8)
            final_image = Image.fromarray(image_np_uint8[0])

            print("--- Proses Generate Gambar Selesai ---")
            return final_image
    except Exception as e:
        print(f"!!! TERJADI ERROR SAAT GENERATE GAMBAR: {e} !!!")
        import traceback
        traceback.print_exc()
        return None

# Hanya jalankan Gradio jika setup berhasil
if unet_model:
    print("\nMenyiapkan antarmuka Gradio...")
    with gr.Blocks(css="footer {display: none !important}") as demo:
        with gr.Row():
            with gr.Column(scale=2):
                positive_prompt = gr.Textbox(label="Positive Prompt", lines=3, placeholder="Contoh: a cinematic photo of a majestic lion...")
                negative_prompt = gr.Textbox(label="Negative Prompt", lines=3, value="blurry, low quality, static, frame, text, watermark")
                with gr.Row():
                    sampler_name = gr.Dropdown(label="Sampler", choices=samplers.KSampler.SAMPLERS, value="euler_ancestral")
                    scheduler = gr.Dropdown(label="Scheduler", choices=samplers.KSampler.SCHEDULERS, value="normal")
                with gr.Row():
                    width = gr.Slider(label="Width", minimum=256, maximum=1920, value=1280, step=64)
                    height = gr.Slider(label="Height", minimum=256, maximum=1080, value=768, step=64)
                with gr.Row():
                    steps = gr.Slider(label="Steps", minimum=1, maximum=50, value=20, step=1)
                    cfg = gr.Slider(label="CFG Scale", minimum=0.0, maximum=15.0, value=3.5, step=0.5)
                seed = gr.Number(label="Seed (-1 untuk acak)", value=-1)
            with gr.Column(scale=1):
                output_image = gr.Image(label="Generated Image", type="pil", format="png")
                generate_button = gr.Button("Generate", variant="primary")

        inputs = [positive_prompt, negative_prompt, width, height, steps, cfg, seed, sampler_name, scheduler]
        generate_button.click(fn=generate_image, inputs=inputs, outputs=[output_image], api_name="generate")

    print("\nMeluncurkan antarmuka Gradio...")
    demo.launch(share=True, inline=False, debug=True)
else:
    print("\nAntarmuka Gradio tidak diluncurkan karena terjadi error pada saat setup model.")